In [14]:
X = ["Hi How are you", "I'll call you later..!", "That's gonna be okay", "see you later"]
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
model = vect.fit(X)
vect.get_feature_names()
trans_model = vect.transform(X)
trans_model.toarray()
#print(trans_model)

array([[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]], dtype=int64)

In [15]:
import pandas as pd
pd.DataFrame(trans_model.toarray(), columns = vect.get_feature_names())

,are,be,call,gonna,hi,how,later,ll,okay,see,that,you
0,1,0,0,0,1,1,0,0,0,0,0,1
1,0,0,1,0,0,0,1,1,0,0,0,1
2,0,1,0,1,0,0,0,0,1,0,1,0
3,0,0,0,0,0,0,1,0,0,1,0,1


In [21]:
sms = pd.read_table('C:\\Users\\Acer\\Desktop\\SMSSpamCollection', header=None, names = ['label','Message'])
print(sms.head())
print(sms.shape)

  label                                            Message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
(5572, 2)


# Lets change ham for 0 and spam for 1

In [24]:
sms['label_new'] = sms.label.map({'ham':0,'spam':1})
print(sms.head())
# How many hams are there and how many spams are there?
sms.label.value_counts()

  label                                            Message  label_new
0   ham  Go until jurong point, crazy.. Available only ...          0
1   ham                      Ok lar... Joking wif u oni...          0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...          1
3   ham  U dun say so early hor... U c already then say...          0
4   ham  Nah I don't think he goes to usf, he lives aro...          0


ham     4825
spam     747
Name: label, dtype: int64

In [28]:
# we will now split the data into X and y and then to train and test datasets
X = sms.Message
y = sms.label_new
print(X.shape)
print(y.shape)

(5572,)
(5572,)


In [31]:
# dividing the data into train and test datasets
from sklearn.cross_validation import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size = 0.25, random_state = 5)
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [36]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(Xtrain)
xtrans = vect.transform(Xtrain)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [40]:
xtest_dtm = vect.transform(Xtest)

In [41]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(xtrans, ytrain)

Wall time: 4 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [45]:
y_pred = nb.predict(xtest_dtm)
from sklearn import metrics
print(metrics.accuracy_score(ytest,y_pred) * 100)

99.1385498923


# Lets build confussion matrix

In [52]:
metrics.confusion_matrix(ytest,y_pred)

array([[1203,    2],
       [  10,  178]])

In [59]:
print(Xtest[Xtest[(y_pred == 1) & (ytest == 0)]])

Message
I (Career Tel) have added u as a contact on INDYAROCKS.COM to send FREE SMS. To remove from phonebook - sms NO to  &lt;#&gt;    NaN
Hey...Great deal...Farm tour 9am to 5pm $95/pax, $50 deposit by 16 May                                                          NaN
Name: Message, dtype: object


In [86]:
y_prob = nb.predict_proba(xtest_dtm)[:,1]
print(metrics.roc_auc_score(ytest,y_prob) * 100)

98.2418116006


# Lets compare the naive bayes to Logistic Regression

In [74]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(xtrans, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [149]:
#lets predict the model accuracy
y_pred = logreg.predict(xtest_dtm)
print(metrics.accuracy_score(ytest,y_pred) * 100)
y_pred_prob = logreg.predict_proba(xtest_dtm)[:,1]
print(metrics.roc_auc_score(ytest,y_pred_prob) * 100)

98.3488872936
99.026220535


# Lets check the haminess or the spaminess of each word | token

In [150]:
word = vect.get_feature_names()
print(len(word))
# check first 10 words
word[0:10]
# check last 10 words
word[-10:]

7427


['yuou', 'yup', 'yupz', 'zed', 'zhong', 'zoe', 'zoom', 'zouk', 'èn', '〨ud']

In [96]:
nb.feature_count_ # check how many times first word appears in class 0 and class 1

array([[  0.,   0.,   1., ...,   0.,   1.,   1.],
       [  6.,  22.,   0., ...,   1.,   0.,   0.]])

In [105]:
ham_vec = nb.feature_count_[0]
spam_vec = nb.feature_count_[1]
import pandas as pd
df = pd.DataFrame({'word': word, 'ham': ham_vec, 'spam': spam_vec})
print(df[0:10])

   ham  spam          word
0  0.0   6.0            00
1  0.0  22.0           000
2  1.0   0.0        000pes
3  0.0   2.0  008704050406
4  0.0   1.0          0089
5  0.0   1.0   01223585236
6  0.0   1.0   01223585334
7  1.0   0.0    0125698789
8  0.0   8.0            02
9  0.0   2.0          0207


In [110]:
df.sample(5,random_state = 6)

,ham,spam,word
2076,2.0,0.0,dancing
5125,0.0,2.0,ppm
6856,2.0,0.0,unfortunately
1045,3.0,0.0,art
2239,0.0,2.0,dining


In [125]:
#Add 1 to both spam and ham
df['ham'] = df.ham + 1
df['spam'] = df.spam + 1
df.sample(5,random_state = 6)

,ham,spam,word
2076,3.0,1.0,dancing
5125,1.0,3.0,ppm
6856,3.0,1.0,unfortunately
1045,4.0,1.0,art
2239,1.0,3.0,dining


In [126]:
# Lets calculate the frequency of each word as being spam or ham
df['ham'] = df.ham / nb.class_count_[0]
df['spam'] = df.spam / nb.class_count_[1]
df.sample(5,random_state = 6)

,ham,spam,word
2076,0.000829,0.001789,dancing
5125,0.000276,0.005367,ppm
6856,0.000829,0.001789,unfortunately
1045,0.001105,0.001789,art
2239,0.000276,0.005367,dining


In [146]:
df['spam_ratio'] = (df.spam / df.ham)
df.sample(20,random_state = 6)

,ham,spam,word,spam_ratio
2076,0.000829,0.001789,dancing,2.158617
5125,0.000276,0.005367,ppm,19.427549
6856,0.000829,0.001789,unfortunately,2.158617
1045,0.001105,0.001789,art,1.618962
2239,0.000276,0.005367,dining,19.427549
5882,0.000552,0.001789,showrooms,3.237925
5014,0.011326,0.008945,place,0.789738
2852,0.000552,0.001789,fox,3.237925
3280,0.000276,0.005367,herself,19.427549
256,0.000276,0.042934,10p,155.420394


In [147]:
df.sort_values('spam_ratio', ascending=False)

,ham,spam,word,spam_ratio
1758,0.000276,0.150268,claim,543.971377
5187,0.000276,0.125224,prize,453.309481
293,0.000276,0.112701,150p,407.978533
6681,0.000276,0.087657,tone,317.316637
309,0.000276,0.073345,18,265.509839
3132,0.000276,0.071556,guaranteed,259.033989
304,0.000276,0.067979,16,246.082290
519,0.000276,0.064401,500,233.130590
246,0.000276,0.059034,1000,213.703041
1129,0.000276,0.051878,awarded,187.799642
